In [1]:
from pyspark import SparkConf, SparkContext


In [2]:
conf = SparkConf().setMaster("local").setAppName("DegreesOfSeparation")
sc = SparkContext(conf = conf)


In [3]:
start_id = 5306 # Spiderman
target_id = 14 # ADAM


In [6]:
# set initial value of shared counter to 0
# used to signal when we found our target character
counter = sc.accumulator(0)


In [7]:
# splits the data into BFS format
def convert_to_bfs(line):
    fields = line.split(" ")
    hero_id = fields[0]
    connections = fields[1:]
    color = "WHITE"
    distance = 9999
    
    if hero_id == start_id:
        color = "GRAY"
        distance = 0
    
    return (hero_id, (connections, distance, color))


In [8]:
# reads input file and call convert_to_bfs() function
def create_starting_rdd():
    input_file = sc.textFile("Marvel-Graph")
    return input_file.map(convert_to_bfs)


In [11]:
def bfs_map(node):
    hero_id = node[0]
    connections = node[1][0]
    distance = node[1][1]
    color = node[1][2]
    results = []
    
    # if the node needs to be expanded
    if color == "GRAY":
        for connection in connections:
            new_hero_id = connection
            new_distance = distance + 1
            new_color = "GRAY"
            
            if target_id == connection:
                counter.add(1)
            
            new_entry = (new_hero_id, ([], new_distance, new_color))
            results.append(new_entry)
        
        # node has ben processed
        color = "BLACK"
    
    # emit the input node
    results.append((hero_id, (connections, distance, color)))
    return results


In [9]:
rdd = create_starting_rdd()


In [ ]:
for iteration in range(0, 10):
    print("Running BFS iteration:", iteration + 1)
    
    mapped = rdd.flatMap(bfs_map)
    
    print("Processing", mapped.count(), "values")
    
    if counter.value > 0:
        print("Hit the target character")
        break